# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118617e+02     1.517525e+00
 * time: 0.09008097648620605
     1     1.101036e+01     1.070636e+00
 * time: 0.9408318996429443
     2    -1.156777e+01     1.088003e+00
 * time: 1.0635600090026855
     3    -3.399999e+01     9.050603e-01
 * time: 1.2219820022583008
     4    -4.757070e+01     7.218841e-01
 * time: 1.3698399066925049
     5    -5.701618e+01     2.472814e-01
 * time: 1.51802396774292
     6    -5.987285e+01     1.698315e-01
 * time: 1.6482160091400146
     7    -6.095889e+01     6.266220e-02
 * time: 1.7552199363708496
     8    -6.137227e+01     6.752112e-02
 * time: 1.8681230545043945
     9    -6.161433e+01     5.777288e-02
 * time: 1.9739210605621338
    10    -6.180671e+01     2.645564e-02
 * time: 2.0875649452209473
    11    -6.195134e+01     2.286778e-02
 * time: 2.199079990386963
    12    -6.202588e+01     1.663470e-02
 * time: 2.3052890300750732
    13    -6.208741e+01     1.573934e-02
 * time: 2.418711900711

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557723
    AtomicNonlocal      14.8522681
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666459916
